In [2]:
# util 
import os
import pickle
import numpy as np
import networkx as nx
from tqdm import tqdm
from collections import defaultdict

import torch
import torch.nn.functional as F
import torch.nn as nn

from typing import Optional, Callable, Tuple, Union

from torch import Tensor
from torch.nn import Parameter

from torch_sparse import SparseTensor, set_diag

from torch_geometric.nn.conv import MessagePassing, NNConv, CGConv, GINEConv
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.inits import reset, zeros, glorot
from torch_geometric.nn.aggr import Aggregation
from torch_geometric.typing import Adj, OptPairTensor, OptTensor, Size

from torch_geometric.nn import global_mean_pool, global_add_pool
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader, NeighborLoader, ClusterData, ClusterLoader
from torch_geometric.utils import from_networkx, to_networkx, add_self_loops, remove_self_loops, softmax

from IPython.display import clear_output

PROJECT_FOLDER = "/dfs/user/sttruong/DucWorkspace/graph_regression"
DATA_FOLDER = PROJECT_FOLDER + "/data"

seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# random.seed(seed)
torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic=True
# torch.backends.cudnn.benchmark = False

In [3]:
def construct_bigLITTLE_graph(data_folder, no_duplicate=False, unobserved=0.0, unobserved_edge=0.0):
    list_files = os.listdir(data_folder)
    list_files = list(filter(lambda x: x.endswith(".pkl"), list_files))
    list_labels = pickle.load(open(PROJECT_FOLDER + "/labels.pkl", "rb"))
    
    dict_graph_size = defaultdict(lambda: [])
    set_cycle_size = set([])
    set_branch_size = set([])
    
    # Directed or Undirected? Edge weights?
    bigLITTLE_graph = nx.DiGraph()
    gid = 0
    
    for gf in list_files:
        idx, cycle_size, branch_size, _ = gf.split("_")
        cycle_size = int(cycle_size)
        branch_size = int(branch_size)
        
        if no_duplicate and len(dict_graph_size[(cycle_size, branch_size)]) > 0:
            continue
            
        # graph = nx.read_gpickle(os.path.join(DATA_FOLDER, gf))
        
        # For testing
        # if cycle_size > 5 or branch_size > 2:
        #     continue
        
        bigLITTLE_graph.add_node(gid, features=[float(cycle_size), float(branch_size)], label=list_labels[int(idx)])
        dict_graph_size[(cycle_size, branch_size)].append(gid)
        
        set_cycle_size.add(cycle_size)
        set_branch_size.add(branch_size)
        gid += 1
    # print(dict_graph_size)
    # Undirected assumption?
    # Same cycle_size & branch_size ==> Edge: [0,0]
    # for k, items in dict_graph_size.items():
    #     for item_idx1 in range(len(items)):
    #         for item_idx2 in range(item_idx1+1, len(items)):
    #             bigLITTLE_graph.add_edge(items[item_idx1], items[item_idx2], e=[0,0])
                
    # Same cycle size & Different branch size ==> Edge: [0, 1]
    # Filter lists of same cycle_size
    for cs in set_cycle_size:
        list_same_cycle_size = list(filter(lambda x: x[0]==cs, dict_graph_size.keys()))
        list_same_cycle_size = list(sorted(list_same_cycle_size, key=lambda x:x[1]))
        # print(list_same_cycle_size)
        for bs_idx1 in range(len(list_same_cycle_size)-1):
            bs_idx2 = bs_idx1 + 1
            key_cb1 = list_same_cycle_size[bs_idx1] # e.g. (3, 1)
            key_cb2 = list_same_cycle_size[bs_idx2] # e.g. (3, 2)

            for gid1 in dict_graph_size[key_cb1]:
                for gid2 in dict_graph_size[key_cb2]:
                    # print(key_cb2[1]-key_cb1[1])
                    bigLITTLE_graph.add_edge(gid1, gid2, e=[0,1])
                    bigLITTLE_graph.add_edge(gid2, gid1, e=[0,-1])
                        
    # Different cycle size & Same branch size ==> Edge: [1, 0]
    for bs in set_branch_size:
        list_same_branch_size = list(filter(lambda x: x[1]==bs, dict_graph_size.keys()))
        list_same_branch_size = list(sorted(list_same_branch_size, key=lambda x:x[0]))
        
        for cs_idx1 in range(len(list_same_branch_size)-1):
            cs_idx2 = cs_idx1 + 1
            key_cb1 = list_same_branch_size[cs_idx1] # e.g. (3, 1)
            key_cb2 = list_same_branch_size[cs_idx2] # e.g. (4, 1)

            for gid1 in dict_graph_size[key_cb1]:
                for gid2 in dict_graph_size[key_cb2]:
                    bigLITTLE_graph.add_edge(gid1, gid2, e=[1,0])
                    bigLITTLE_graph.add_edge(gid2, gid1, e=[-1,0])
                    
    
    # Add all other edge as [0,0]
    list_nodes = list(bigLITTLE_graph.nodes)
    for i in range(len(list_nodes)):
        for j in range(i, len(list_nodes)):
            nid_i = list_nodes[i]
            nid_j = list_nodes[j]
            if not bigLITTLE_graph.has_edge(nid_i, nid_j):
                bigLITTLE_graph.add_edge(gid1, gid2, e=[0,0])
                bigLITTLE_graph.add_edge(gid2, gid1, e=[0,0])
    
    if unobserved > 0:
        unobserved_node_idxs = np.random.choice(list(range(bigLITTLE_graph.number_of_nodes())), 
                                size=int(unobserved*bigLITTLE_graph.number_of_nodes()), 
                                replace=False)
    else:
        unobserved_node_idxs = None
    
    if unobserved_edge > 0:
        # Remove unobserved edges
        num_edge_to_remove = int(bigLITTLE_graph.number_of_edges() * unobserved_edge)
            
        list_edges = np.array([list(e) for e in bigLITTLE_graph.edges])
        list_unique_edges = list_edges[list_edges[:,1] >= list_edges[:,0]]
        edge_idxs = np.random.choice([0,1], 
                                size=list_unique_edges.shape[0], 
                                p=[unobserved_edge, 1-unobserved_edge])
        list_remove_edges = list_unique_edges[edge_idxs==0]
        
        for edge in list_remove_edges:
            u, v = edge
            bigLITTLE_graph.remove_edge(u,v)
            bigLITTLE_graph.remove_edge(v,u)
    
    return bigLITTLE_graph, unobserved_node_idxs

In [4]:
def get_edge_color(e):
    if e == [0,1]:
        return "black"
    elif e == [0,-1]:
        return "red"
    elif e == [1,0]:
        return "green"
    elif e == [-1,0]:
        return "blue"
    else:
        return "yellow"
    
def draw_graph(graph):
    nodeLabels = {nid:graph.nodes[nid]["label"] for nid in graph.nodes}
    nodeColors = "grey"
    edgeColor = [get_edge_color(graph.edges[eid]["e"])for eid in graph.edges]

    nx.draw(graph, nx.kamada_kawai_layout(graph), edge_color=edgeColor, width=1, linewidths=0.1,
              node_size=500, node_color=nodeColors, alpha=0.9,
              labels=nodeLabels)
    
def transform_func(graph, device):
    graph.x = graph.x.to(device)
    graph.x_lp = graph.x_lp.to(device)
    graph.y = graph.label.to(device)
    graph.edge_attr = graph.edge_attr.to(device)
    graph.edge_index = graph.edge_index.to(device)
    return graph

In [23]:
class TrickyAggregation(Aggregation):
    '''
    Aggregation for the tricky graph
    This class is used to aggregate the top-k node features of the tricky graph by mean function
    '''
    def __init__(self, k=2):
        super(TrickyAggregation, self).__init__()
        self.k = k

    def forward(self, x: Tensor, index: Optional[Tensor] = None,
                ptr: Optional[Tensor] = None, dim_size: Optional[int] = None,
                dim: int = -2) -> Tensor:
        """
        Get top-k node features by mean function among indexed nodes.
        Using Optimal Transport to get the top-k node features.
        """
        pass

class TrickyNNConv(MessagePassing):
    def __init__(self, 
                 in_channels: Union[int, Tuple[int, int]],
                 out_channels: int, aggr: str = 'mean', 
                 edge_dim: Optional[int] = None,
                 add_self_loops: bool = False, 
                 negative_slope: float = 0.2, 
                 fill_value: Union[float, Tensor, str] = 'mean',
                 dropout: float = 0.0,
                 bias: bool = False, **kwargs):

        if aggr == 'tricky':
            kwargs['aggr'] = TrickyAggregation()
        else:
            kwargs['aggr'] = aggr

        super().__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.edge_dim = edge_dim
        self.add_self_loops = add_self_loops
        self.negative_slope = negative_slope
        self.fill_value = fill_value
        self.dropout = dropout

        if isinstance(in_channels, int):
            in_channels = (in_channels, in_channels)

        
        if isinstance(in_channels, int):
            self.lin_src = Linear(in_channels, out_channels,
                                  bias=False, weight_initializer='glorot')
            self.lin_dst = self.lin_src
        else:
            self.lin_src = Linear(in_channels[0], out_channels, False,
                                  weight_initializer='glorot')
            self.lin_dst = Linear(in_channels[1], out_channels, False,
                                  weight_initializer='glorot')
            
        # The learnable parameters to compute attention coefficients:
        # self.att_src = Parameter(torch.Tensor(1, out_channels))
        # self.att_dst = Parameter(torch.Tensor(1, out_channels))

        if edge_dim is not None:
            self.att_edge = Parameter(torch.Tensor(1, out_channels))
        else:
            self.register_parameter('att_edge', None)    
        
        self.nn = Linear(out_channels, out_channels, bias=False, weight_initializer='uniform')

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        reset(self.nn)
        
        self.lin_src.reset_parameters()
        self.lin_dst.reset_parameters()
        
        # glorot(self.att_src)
        # glorot(self.att_dst)
        glorot(self.att_edge)
        zeros(self.bias)

    def forward(self, x: Union[Tensor, OptPairTensor], edge_index: Adj,
                edge_attr: OptTensor = None, size: Size = None) -> Tensor:
        """"""
        if isinstance(x, Tensor):
            assert x.dim() == 2, "Static graphs not supported"
            x_src = x_dst = self.lin_src(x)
        else:  # Tuple of source and target node features:
            x_src, x_dst = x
            assert x_src.dim() == 2, "Static graphs not supported"
            x_src = self.lin_src(x_src)
            if x_dst is not None:
                x_dst = self.lin_dst(x_dst)
                
        x = (x_src, x_dst)

        # Next, we compute node-level attention coefficients, both for source
        # and target nodes (if present):
        # alpha_src = (x_src * self.att_src).sum(dim=-1).view(1, -1)
        # alpha_dst = None if x_dst is None else (x_dst * self.att_dst).sum(-1).view(1, -1)
        # alpha = (alpha_src, alpha_dst)
        alpha = x
        
        if self.add_self_loops:
            if isinstance(edge_index, Tensor):
                # We only want to add self-loops for nodes that appear both as
                # source and target nodes:
                num_nodes = x_src.size(0)
                if x_dst is not None:
                    num_nodes = min(num_nodes, x_dst.size(0))
                num_nodes = min(size) if size is not None else num_nodes
                edge_index, edge_attr = remove_self_loops(
                    edge_index, edge_attr)
                edge_index, edge_attr = add_self_loops(
                    edge_index, edge_attr, fill_value=self.fill_value,
                    num_nodes=num_nodes)
            elif isinstance(edge_index, SparseTensor):
                if self.edge_dim is None:
                    edge_index = set_diag(edge_index)
                else:
                    raise NotImplementedError(
                        "The usage of 'edge_attr' and 'add_self_loops' "
                        "simultaneously is currently not yet supported for "
                        "'edge_index' in a 'SparseTensor' form")

        # edge_updater_type: (alpha: OptPairTensor, edge_attr: OptTensor)
        alpha = self.edge_updater(edge_index, alpha=alpha, edge_attr=edge_attr)
        
        # propagate_type: (x: OptTensor, alpha: Tensor, edge_attr: OptTensor)
        out = self.propagate(edge_index, x=x, edge_attr=edge_attr, alpha=alpha, size=size)

        if self.bias is not None:
            out = out + self.bias
            
        return out

    def edge_update(self, alpha_j: Tensor, alpha_i: OptTensor,
                    edge_attr: OptTensor, index: Tensor, ptr: OptTensor,
                    size_i: Optional[int]) -> Tensor:
        # Given edge-level attention coefficients for source and target nodes,
        # we simply need to sum them up to "emulate" concatenation:
        # alpha = alpha_j if alpha_i is None else alpha_j + alpha_i

        if edge_attr is not None:
            if edge_attr.dim() == 1:
                edge_attr = edge_attr.view(-1, 1)
            edge_attr = edge_attr.view(-1, self.out_channels)
            alpha_edge = (edge_attr * self.att_edge).sum(dim=-1)
            alpha = alpha_edge ** 2

        # alpha = F.leaky_relu(alpha, self.negative_slope)
        # alpha = (alpha * self.att_edge).sum(dim=-1)
        alpha = softmax(alpha, index, ptr, size_i)
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        return alpha
    
    def message(self, x_j: Tensor, edge_attr: Tensor, alpha: Tensor) -> Tensor:
        weight = edge_attr.view(-1, self.out_channels)
        return alpha.unsqueeze(-1) * self.nn(weight + x_j).squeeze(1) # torch.matmul(x_j.unsqueeze(1), weight).squeeze(1)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.in_channels}, '
                f'{self.out_channels}, aggr={self.aggr}, nn={self.nn})')

In [24]:
class GNN(nn.Module):
    def __init__(self, node_channels, edge_channels, hidden_channels):
        super(GNN, self).__init__()

        self.edge_embed = nn.Sequential(
                Linear(edge_channels, hidden_channels, bias=False,
                                   weight_initializer='glorot')
                # nn.Linear(hidden_channels, hidden_channels, bias=True)
            )
        
        self.conv1 = TrickyNNConv(node_channels, hidden_channels, aggr="add", edge_dim=hidden_channels)
        self.conv1_e = nn.Sequential(
                Linear(hidden_channels, hidden_channels, bias=False,
                                   weight_initializer='glorot')
                # nn.Linear(hidden_channels, hidden_channels, bias=True)
            )
        
        self.conv2 = TrickyNNConv(hidden_channels, hidden_channels, aggr="add", edge_dim=hidden_channels)
        
        self.lin1 = nn.Linear(hidden_channels, hidden_channels, bias=True)
        self.lin2 = nn.Linear(hidden_channels, hidden_channels, bias=True)
        self.lin3 = nn.Linear(hidden_channels, 1, bias=True)
        
        # Max aggregating, Top-k mean, sorted + weighted sum
        # Increase GNN layers

    def forward(self, x, edge_index, edge_attr, batch):
        
        '''
        edge_attr: batch_size * 2
        '''
        # 1. Obtain node embeddings
        # x = self.node_embed(x)
        e = self.edge_embed(edge_attr)
        
        z = self.conv1(x=x, edge_index=edge_index, edge_attr=e)
        z = z.relu()
        
        e = self.conv1_e(e)#.relu()
        z = self.conv2(x=z, edge_index=edge_index, edge_attr=e)
        z = z.relu()
        
        # 2. Apply a final classifier
        z = F.dropout(z, p=0.1, training=True)
        
        z = self.lin1(z)
        z = z.relu()
        z = self.lin2(z)
        z = z.relu()
        
        z = self.lin3(z)
        z = torch.sigmoid(z) * 110
        
        return z
    
class LinkPredictor(nn.Module):
    def __init__(self, node_channels, edge_channels, hidden_channels):
        super(LinkPredictor, self).__init__()
        
        self.node_embed = nn.Linear(node_channels, hidden_channels, bias=False)
        
        self.lin1 = nn.Linear(node_channels, hidden_channels, bias=True)
        self.lin2 = nn.Linear(hidden_channels, edge_channels, bias=True)

    def forward(self, x, observed_edge_index, batch):
        
        '''
        x: (num_nodes, 2)
        observed_edge_nid: (num_observed_edge, 2)
        '''
        # 1. Obtain node embeddings
        z = self.node_embed(x)
        z = z.relu()
        
        # 2. Apply a final classifier
        # z = F.dropout(z, p=0.1, training=True)
        
        # observed_edge_nid: [(1,2), (3,4)] ==> (x[1] <-> x[2])
        # (40.1 30.1) ==> e:[1,0]
        # (50.1 70.1) ==> e:[0,0]
        head = x[observed_edge_index[1]]
        tail = x[observed_edge_index[0]]
        
        e = head - tail
        e = self.lin1(e)
        e = self.lin2(e)
        return e
    

In [25]:
class JointModel(nn.Module):
    def __init__(self, node_channels, edge_channels, hidden_channels):
        super(JointModel, self).__init__()
        
        #=============================
        # LINK PREDICTOR
        #=============================
        self.link_predictor = LinkPredictor(node_channels=node_channels, edge_channels=edge_channels, 
                    hidden_channels=hidden_channels)
        lp_parameters = filter(lambda p: p.requires_grad, self.link_predictor.parameters())
        lp_params = sum([np.prod(p.size()) for p in lp_parameters])
        print(self.link_predictor)
        print("Number of LP parameters: ", lp_params)


        #=============================
        # NODE REGRESSION
        #=============================
        self.gnn = GNN(node_channels=node_channels, edge_channels=edge_channels, 
                    hidden_channels=hidden_channels)
        gnn_parameters = filter(lambda p: p.requires_grad, self.gnn.parameters())
        gnn_params = sum([np.prod(p.size()) for p in gnn_parameters])
        print(self.gnn)
        print("Number of GNN parameters: ", gnn_params)

    def forward(self, x, x_lp, edge_index, batch, edge_attr=None):
        predicted_edge_attr = self.link_predictor(x_lp, edge_index, batch)
        # Stil remaining edge [0,0]
        
        if self.training and edge_attr is not None:
            # TEACHER FORCING
            node_y = self.gnn(x, edge_index, edge_attr, batch)
        else:
            # NON-TEACHER FORCING
            node_y = self.gnn(x, edge_index, predicted_edge_attr, batch)
            
        return predicted_edge_attr, node_y

In [8]:
def train(model, loader, random_mask=0, observed_idxs=None, criterion=None, optimizer=None):
    model.train()
    total_loss = 0
    steps = 0

    # Iterate in batches over the training dataset
    for data in loader:
        edge_attr, node_y = model(data.x, data.x_lp, data.edge_index, data.batch, edge_attr=data.edge_attr)
        
        # Compute the loss for link predictor
        lp_loss = criterion(
            edge_attr, 
            data.edge_attr
        )
        # Compute the loss
        node_loss = criterion(
            node_y[observed_idxs], 
            data.y[observed_idxs].view(-1, 1)
        )
        
        total_loss += lp_loss + node_loss
        total_loss.backward(); 
        optimizer.step(); optimizer.zero_grad(); steps += 1

    return total_loss / steps

def test(model, loader, mc_dropout_sample=100, criterion=None):
    model.eval()
    mse = 0
    steps = 0

    # Iterate in batches over the training/test dataset
    for data in loader:
        node_ys = []
        edge_attrs = []
        
        for _ in range(mc_dropout_sample):
            # NON-TEACHER FORCING
            edge_attr, node_y = model(data.x, data.x_lp, data.edge_index, data.batch)
            
            edge_attrs.append(edge_attr)
            node_ys.append(node_y)
        edge_attrs = torch.stack(edge_attrs).detach().cpu()
        node_ys = torch.stack(node_ys).detach().cpu()
        
        # Check against ground-truth labels
        mse += criterion(edge_attrs.mean(0), data.edge_attr.detach().cpu()) \
            + criterion(node_ys.mean(0), data.y.view(-1, 1).detach().cpu())
        steps += 1
        
        # out_std = out.std(0)
    return mse / steps  # Derive ratio of correct predictions.

In [19]:
generate_data = True
unobserved_fraction = 0.2
device_data = "cuda:7"

if generate_data:
    bL_graph_train, unobserved_idxs = construct_bigLITTLE_graph(DATA_FOLDER, 
                                            unobserved=unobserved_fraction,
                                            unobserved_edge=unobserved_fraction)
    observed_idxs = list(set(bL_graph_train.nodes) - set(unobserved_idxs))
    # draw_graph(bL_graph_train)
    
    data_train = from_networkx(bL_graph_train)
    data_train.x = torch.ones(data_train.label.shape[0], 2).type(torch.FloatTensor)
    data_train.x[:,1] = 0
    data_train.x_lp = data_train.features.type(torch.FloatTensor)
    data_train.edge_attr = data_train.e.type(torch.FloatTensor)
    data_train = transform_func(data_train, device_data)
    
    c_data_train = ClusterData(data_train, num_parts=1)
    train_loader = ClusterLoader(c_data_train)
    
    
    
    bL_graph_test, _ = construct_bigLITTLE_graph(DATA_FOLDER)
    # draw_graph(bL_graph)
    
    data_test = from_networkx(bL_graph_test)
    data_test.x = torch.ones(data_test.label.shape[0], 2).type(torch.FloatTensor)
    data_test.x[:,1] = 0
    data_test.x_lp = data_test.features.type(torch.FloatTensor)
    data_test.edge_attr = data_test.e.type(torch.FloatTensor)
    data_test = transform_func(data_test, device_data)
    
    c_data_test = ClusterData(data_test, num_parts=1)
    test_loader = ClusterLoader(c_data_test)

Computing METIS partitioning...
Done!
Computing METIS partitioning...
Done!


In [20]:
# Test unconnected graph
neighbors_list = []
for nid in bL_graph_train.nodes:
    node_neighbors = bL_graph_train.neighbors(nid)
    node_observed_neighbors = set(node_neighbors) - set(unobserved_idxs)
    node_unobserved_neighbors = set(node_neighbors) - set(node_observed_neighbors)
    neighbors_list.append([nid, len(node_observed_neighbors), len(node_unobserved_neighbors)])
# print(neighbors_list)
neighbors_list = np.array(neighbors_list)
non_neighbor_nids = np.where(neighbors_list[:,1] == 0)[0]
print(non_neighbor_nids)
assert non_neighbor_nids.shape[0] == 0

[]


In [26]:
min_mse = 1e10
min_epoch = 0
epochs = 100000
lr = 0.0005
device = device_data
hidden_channels = 64


model = JointModel(node_channels=2, edge_channels=2, 
            hidden_channels=hidden_channels).to(device)
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(model)
print("Number of parameters: ", params)


optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.L1Loss()

for epoch in tqdm(range(epochs)):
    train_mse = train(model, train_loader, observed_idxs=observed_idxs, criterion=criterion, optimizer=optimizer)
    
    if (epoch+1) % 1000 == 0:
        clear_output(wait=True)
        test_mse = test(model, test_loader, criterion=criterion)
        if test_mse < min_mse:
            min_mse = test_mse
            min_epoch = epoch
        print(f'Epoch: {epoch+1:03d}, Train MAE: {train_mse:.4f},',
              f'Test MAE: {test_mse:.4f}, Min MAE: {min_mse:.4f}')
    else: test_mse = 0

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [37:29<00:00, 44.45it/s]

Epoch: 100000, Train MAE: 0.0355, Test MAE: 0.4429, Min MAE: 0.2805


In [ ]:
torch.save(model.state_dict(), f"/dfs/user/sttruong/DucWorkspace/graph_regression/model.pt")

In [10]:
!nvidia-smi

Tue Nov 22 19:57:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Quadro R...  On   | 00000000:43:00.0 Off |                    0 |
| N/A   36C    P0    57W / 250W |   7166MiB / 45556MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA Quadro R...  On   | 00000000:44:00.0 Off |                    0 |
| N/A   